<div style="width: 100%; font-family: 'Segoe UI', Helvetica, Arial, sans-serif; box-sizing: border-box;">

<div style="
        background: linear-gradient(135deg, #0f2027 0%, #203a43 50%, #2c5364 100%);
        border-radius: 12px;
        padding: 35px;
        text-align: center;
        margin-bottom: 25px;
        border: 2px solid #4A90E2;
        box-shadow: 0 8px 20px rgba(74, 144, 226, 0.3);
        box-sizing: border-box;
    ">
        <h1 style="
            color: #fff;
            margin: 0;
            font-size: 32px;
            font-weight: 300;
            letter-spacing: 4px;
            text-transform: uppercase;
        ">
            AstroSpectro Pipeline
        </h1>
        <h2 style="
            color: #4A90E2;
            margin: 15px 0 0 0;
            font-weight: 300;
            font-style: italic;
            font-size: 18px;
        ">
            Automated Stellar Classification from LAMOST DR5 Spectra
        </h2>
        <div style="
            margin-top: 20px;
            padding-top: 15px;
            border-top: 1px solid rgba(74, 144, 226, 0.3);
        ">
            <span style="color: #95D5F7; font-size: 13px; text-transform: uppercase; letter-spacing: 1px;">
                Master Orchestration Notebook • ML Pipeline • DR5 Catalog
            </span>
        </div>
    </div>

<div style="
        background: linear-gradient(135deg, #1a1a2e 0%, #16213e 100%);
        border: 2px solid #4A90E2;
        border-radius: 12px;
        padding: 25px;
        margin-bottom: 20px;
        box-shadow: 0 4px 15px rgba(0,0,0,0.3);
        box-sizing: border-box;
        display: flex; 
        flex-wrap: wrap; 
        gap: 30px;
    ">
        
<div style="
            flex: 1; 
            min-width: 250px; 
            border-right: 1px solid rgba(74, 144, 226, 0.3); 
            padding-right: 15px;
        ">
            <h4 style="
                color: #4A90E2;
                margin: 0 0 18px 0;
                font-size: 14px;
                text-transform: uppercase;
                letter-spacing: 1px;
                border-bottom: 2px solid rgba(74, 144, 226, 0.4);
                padding-bottom: 8px;
            ">Project Information</h4>
            
<div style="margin-bottom: 12px;">
                <span style="color: #95D5F7; font-size: 11px; text-transform: uppercase; letter-spacing: 0.5px;">Project</span><br>
                <strong style="color: white; font-size: 16px;">AstroSpectro</strong>
            </div>
            
<div style="margin-bottom: 12px;">
                <span style="color: #95D5F7; font-size: 11px; text-transform: uppercase; letter-spacing: 0.5px;">Author</span><br>
                <strong style="color: white; font-size: 16px;">Alex Baker</strong>
            </div>
            
<div style="margin-bottom: 12px;">
                <span style="color: #95D5F7; font-size: 11px; text-transform: uppercase; letter-spacing: 0.5px;">Repository</span><br>
                <a href="https://github.com/PhD-Brown/AstroSpectro" style="color: #4A90E2; font-size: 14px; text-decoration: none;">github.com/PhD-Brown/AstroSpectro</a>
            </div>
            
<div>
                <span style="color: #95D5F7; font-size: 11px; text-transform: uppercase; letter-spacing: 0.5px;">Documentation</span><br>
                <a href="https://phd-brown.github.io/AstroSpectro/" style="color: #4A90E2; font-size: 14px; text-decoration: none;">phd-brown.github.io/AstroSpectro</a>
            </div>
        </div>
        
<div style="flex: 1; min-width: 250px;">
            <h4 style="
                color: #4A90E2;
                margin: 0 0 18px 0;
                font-size: 14px;
                text-transform: uppercase;
                letter-spacing: 1px;
                border-bottom: 2px solid rgba(74, 144, 226, 0.4);
                padding-bottom: 8px;
            ">Technical Stack</h4>
            
<div style="margin-bottom: 12px;">
                <span style="color: #95D5F7; font-size: 11px; text-transform: uppercase; letter-spacing: 0.5px;">Data Source</span><br>
                <strong style="color: white; font-size: 16px;">LAMOST DR5 Catalog</strong>
            </div>
            
<div style="margin-bottom: 12px;">
                <span style="color: #95D5F7; font-size: 11px; text-transform: uppercase; letter-spacing: 0.5px;">ML Framework</span><br>
                <strong style="color: white; font-size: 16px;">scikit-learn + XGBoost</strong>
            </div>
            
<div style="margin-bottom: 12px;">
                <span style="color: #95D5F7; font-size: 11px; text-transform: uppercase; letter-spacing: 0.5px;">Enrichment</span><br>
                <strong style="color: white; font-size: 16px;">Gaia DR3 Cross-matching</strong>
            </div>
            
<div>
                <span style="color: #95D5F7; font-size: 11px; text-transform: uppercase; letter-spacing: 0.5px;">Notebook Version</span><br>
                <strong style="color: white; font-size: 16px;">v2.0 Professional</strong>
            </div>
        </div>
        
</div>

</div>

## Overview & Objectives

<div style="
    background: linear-gradient(135deg, #1a1a2e 0%, #16213e 100%);
    border-left: 5px solid #9B59B6;
    padding: 20px;
    border-radius: 8px;
    margin: 20px 0;
">

**Scientific Context :**  
AstroSpectro is a comprehensive machine learning pipeline designed to automate stellar spectral classification using data from the LAMOST DR5 survey. This notebook orchestrates the entire workflow from raw FITS files to trained classifiers, incorporating Gaia DR3 cross-matching for enhanced astrometric and photometric features.

**Pipeline Objectives :**
1. **Data Management** — Select and track spectral batches with no duplication
2. **Catalog Generation** — Extract metadata and enrich with Gaia DR3 parameters
3. **Feature Engineering** — Compute spectral indices, line measurements, and statistical features
4. **Model Training** — Train Random Forest or XGBoost classifiers with feature selection
5. **Performance Tracking** — Log experiments, save artifacts, and enable reproducibility

**Key Innovation :**  
Unlike traditional spectral analysis workflows, AstroSpectro combines physical spectral features (Balmer lines, Ca II H/K, metallicity proxies) with machine learning for robust, interpretable stellar classification.

</div>

<div style="
    background: linear-gradient(135deg, #1a1a2e 0%, #16213e 100%);
    border-left: 5px solid #E67E22;
    padding: 20px;
    border-radius: 8px;
    margin: 20px 0;
">

**⚠️ Important Notes :**
- **Gaia Authentication** : Requires valid GAIA_USER and GAIA_PASS in `.env` file for bulk mode
- **Data Directory** : Ensure LAMOST DR5 FITS files are downloaded to `RAW_DATA_DIR`
- **Computational Resources** : Large batches (>10k spectra) may require substantial RAM
- **Network** : Gaia cross-matching is network-intensive; use stable connection

</div>

<div style="height: 40px;"></div>

<hr style="
    border: 0;
    height: 3px;
    background: linear-gradient(90deg, transparent, #4A90E2, transparent);
    border-radius: 2px;
">

<div style="height: 40px;"></div>

<h2 id="table-of-contents">Table of Contents</h2>

<div style="
    width: 100%;
    box-sizing: border-box;
    background: linear-gradient(135deg, #1a1a2e 0%, #16213e 100%);
    padding: 25px;
    border-radius: 12px;
    color: white;
">

<div style="display: flex; flex-wrap: wrap; gap: 20px; justify-content: space-between;">
        
<div style="flex: 1 1 45%; min-width: 300px; background: rgba(74, 144, 226, 0.08); padding: 20px; border-radius: 8px; border-left: 4px solid #4A90E2; box-sizing: border-box;">
            <strong style="color: #4A90E2; font-size: 16px; display: block; margin-bottom: 10px;">▸ Setup & Configuration</strong>
            <ul style="margin: 0; padding-left: 20px; line-height: 1.8;">
                <li><a href="#setup" style="color: #95D5F7; text-decoration: none;">Environment Setup</a></li>
                <li><a href="#imports" style="color: #95D5F7; text-decoration: none;">Dependencies & Imports</a></li>
                <li><a href="#gaia-connect" style="color: #95D5F7; text-decoration: none;">Gaia Authentication</a></li>
            </ul>
        </div>

<div style="flex: 1 1 45%; min-width: 300px; background: rgba(155, 89, 182, 0.08); padding: 20px; border-radius: 8px; border-left: 4px solid #9B59B6; box-sizing: border-box;">
            <strong style="color: #9B59B6; font-size: 16px; display: block; margin-bottom: 10px;">▸ Quick Start</strong>
            <ul style="margin: 0; padding-left: 20px; line-height: 1.8;">
                <li><a href="#full-pipeline" style="color: #C39BD3; text-decoration: none;">Run Full Pipeline</a></li>
                <li><a href="#interactive-ui" style="color: #C39BD3; text-decoration: none;">Interactive Training UI</a></li>
            </ul>
        </div>

<div style="flex: 1 1 45%; min-width: 300px; background: rgba(46, 204, 113, 0.08); padding: 20px; border-radius: 8px; border-left: 4px solid #2ECC71; box-sizing: border-box;">
            <strong style="color: #2ECC71; font-size: 16px; display: block; margin-bottom: 10px;">▸ Data Pipeline</strong>
            <ul style="margin: 0; padding-left: 20px; line-height: 1.8;">
                <li><a href="#step-1" style="color: #82E0AA; text-decoration: none;">Step 1: Download Spectra</a></li>
                <li><a href="#step-2" style="color: #82E0AA; text-decoration: none;">Step 2: Batch Selection</a></li>
                <li><a href="#step-3" style="color: #82E0AA; text-decoration: none;">Step 3: Catalog & Gaia Enrichment</a></li>
                <li><a href="#step-4" style="color: #82E0AA; text-decoration: none;">Step 4: Feature Extraction</a></li>
            </ul>
        </div>

<div style="flex: 1 1 45%; min-width: 300px; background: rgba(230, 126, 34, 0.08); padding: 20px; border-radius: 8px; border-left: 4px solid #E67E22; box-sizing: border-box;">
            <strong style="color: #E67E22; font-size: 16px; display: block; margin-bottom: 10px;">▸ Machine Learning</strong>
            <ul style="margin: 0; padding-left: 20px; line-height: 1.8;">
                <li><a href="#step-5" style="color: #F5B041; text-decoration: none;">Step 5: Model Training</a></li>
                <li><a href="#evaluation" style="color: #F5B041; text-decoration: none;">Performance Evaluation</a></li>
                <li><a href="#artifacts" style="color: #F5B041; text-decoration: none;">Artifacts & Logging</a></li>
            </ul>
        </div>

</div>
</div>

<div style="height: 40px;"></div>

<hr style="
    border: 0;
    height: 3px;
    background: linear-gradient(90deg, transparent, #4A90E2, transparent);
    border-radius: 2px;
">

<div style="height: 40px;"></div>

<a id="setup"></a>

# Step 0 : Environment Setup

<div style="
    background: linear-gradient(135deg, #1a1a2e 0%, #16213e 100%);
    border-left: 5px solid #4A90E2;
    padding: 20px;
    border-radius: 8px;
    margin: 20px 0;
">

**Purpose :**  
Initialize the project environment, validate directory structure, and prepare the `MasterPipeline` orchestrator.

**What happens here :**
- Automatic creation/validation of directory tree (`RAW_DATA_DIR`, `CATALOG_DIR`, `PROCESSED_DIR`, `MODELS_DIR`, `REPORTS_DIR`)
- Loading of environment variables from `.env` (Gaia credentials)
- Instantiation of the `MasterPipeline` object
- Optional: Gaia archive connection test

**Expected Output :**
- A ready-to-use `pipeline` object
- Confirmation messages about project root and Gaia connection status
- System information (Python version, NumPy version, platform)

</div>

<a id="imports"></a>

## Dependencies & Imports

In [ ]:
# ============================================================================
# ASTROSPECTRO PIPELINE — IMPORTS & CONFIGURATION
# ============================================================================
"""
Master orchestration notebook for the AstroSpectro ML pipeline.
This cell imports all necessary modules and configures the runtime environment.

Dependencies:
- Pipeline modules: master.py, classifier.py, preprocessor.py, feature_engineering.py
- External: astroquery (Gaia), astropy, scikit-learn, xgboost
- Utilities: pandas, numpy, matplotlib
"""

# Core utilities
import sys
import platform
from pathlib import Path

# Scientific computing
import numpy as np
import pandas as pd

# Visualization
import matplotlib.pyplot as plt
import matplotlib

# AstroSpectro modules
from utils import setup_project_env, load_env_vars
from pipeline.master import MasterPipeline
from pipeline.classifier import SpectralClassifier

# Astronomy tools
from astroquery.gaia import Gaia

# ============================================================================
# SYSTEM INFORMATION
# ============================================================================

print("═" * 70)
print("   ASTROSPECTRO PIPELINE — INITIALIZATION SUCCESSFUL")
print("═" * 70)
print(f"   Python Version     : {platform.python_version()}")
print(f"   NumPy Version      : {np.__version__}")
print(f"   Pandas Version     : {pd.__version__}")
print(f"   Matplotlib Version : {matplotlib.__version__}")
print(f"   Platform           : {platform.system()} {platform.release()}")
print(f"   Architecture       : {platform.machine()}")
print("═" * 70)
print("   Status: Ready for pipeline execution")
print("═" * 70)

<a id="gaia-connect"></a>

## Gaia Archive Authentication

In [ ]:
# ============================================================================
# PROJECT ENVIRONMENT SETUP
# ============================================================================

# Initialize project directories
print("\nInitializing project environment...\n")
paths = setup_project_env()

# Display directory structure
print("\nDirectory Structure:")
print("─" * 50)
for key, path in paths.items():
    print(f"   {key:20s} → {path}")
print("─" * 50)

# Load environment variables
env_vars = load_env_vars()

# ============================================================================
# GAIA ARCHIVE CONNECTION
# ============================================================================

print("\nAttempting Gaia Archive authentication...\n")

gaia_connected = False
try:
    Gaia.login(
        user=env_vars.get("GAIA_USER"),
        password=env_vars.get("GAIA_PASS")
    )
    gaia_connected = True
    print("Gaia authentication successful")
    print("   → Bulk cross-matching available")
    print("   → Full DR3 catalog access enabled\n")
except Exception as e:
    print(f"Gaia authentication failed: {e}")
    print("   → Running in LOCAL mode (no Gaia enrichment)")
    print("   → To enable Gaia: check credentials in .env file\n")

# ============================================================================
# MASTERPIPELINE INSTANTIATION
# ============================================================================

print("Instantiating MasterPipeline orchestrator...\n")

pipeline = MasterPipeline(
    raw_data_dir=paths["RAW_DATA_DIR"],
    catalog_dir=paths["CATALOG_DIR"],
    processed_dir=paths["PROCESSED_DIR"],
    models_dir=paths["MODELS_DIR"],
    reports_dir=paths["REPORTS_DIR"],
)

print("═" * 70)
print("   SETUP COMPLETE — PIPELINE READY FOR EXECUTION")
print("═" * 70)
print(f"   Gaia Status        : {'🟢 Connected' if gaia_connected else '🔴 Disconnected'}")
print(f"   Interactive UI     : Available via pipeline.interactive_training_runner()")
print(f"   Full Pipeline      : Available via pipeline.run_full_pipeline()")
print("═" * 70)

<div style="height: 50px;"></div>

<hr style="
    border: 0;
    height: 3px;
    background: linear-gradient(90deg, transparent, #9B59B6, transparent);
    border-radius: 2px;
">

<div style="height: 50px;"></div>

<a id="full-pipeline"></a>

# Quick Start : Full Pipeline Execution

<div style="
    background: linear-gradient(135deg, #1a1a2e 0%, #16213e 100%);
    border-left: 5px solid #9B59B6;
    padding: 20px;
    border-radius: 8px;
    margin: 20px 0;
">

**One-Shot Pipeline :**  
Execute the **complete A→Z workflow** in a single call :

```
Batch Selection → Catalog Generation → Gaia Enrichment → Feature Extraction → Model Training
```

**Recommended Starting Parameters :**
- `batch_size` : 200–500 spectra (for quick testing), 5000–10000 (for production)
- `model_type` : "RandomForest" (faster) or "XGBoost" (more accurate)
- `n_estimators` : 200–400 for Random Forest, 100–200 for XGBoost
- `enrich_gaia` : Set to `True` when Gaia connection is stable

**Output :**
- Trained model saved to `MODELS_DIR`
- Training report (JSON) saved to `REPORTS_DIR`
- Feature importance analysis
- Performance metrics (accuracy, confusion matrix, classification report)

</div>

In [ ]:
# ============================================================================
# FULL PIPELINE EXECUTION — One-Shot Workflow
# ============================================================================

pipeline.run_full_pipeline(
    # ─────────────────────────────────────────────────────────────────────
    # DATA SELECTION
    # ─────────────────────────────────────────────────────────────────────
    batch_size=500,                    # Number of spectra to process
    
    # ─────────────────────────────────────────────────────────────────────
    # MODEL CONFIGURATION
    # ─────────────────────────────────────────────────────────────────────
    model_type="RandomForest",         # "RandomForest" or "XGBoost"
    n_estimators=200,                  # Number of trees
    prediction_target="main_class",    # "main_class", "sub_class_top25", "sub_class_bins"
    
    # ─────────────────────────────────────────────────────────────────────
    # GAIA ENRICHMENT
    # ─────────────────────────────────────────────────────────────────────
    enrich_gaia=False,                 # Enable Gaia DR3 cross-matching
    # mode='bulk',                     # Uncomment if enrich_gaia=True
    # include_risky=False,             # Experimental: radius/mass/age
    # ruwe_max=1.4,                    # Quality filter for astrometry
    
    # ─────────────────────────────────────────────────────────────────────
    # OUTPUT & LOGGING
    # ─────────────────────────────────────────────────────────────────────
    save_and_log=True,                 # Save model + generate report
)

<div style="height: 30px;"></div>

<a id="interactive-ui"></a>

## Interactive Training Interface

<div style="
    background: linear-gradient(135deg, #1a1a2e 0%, #16213e 100%);
    border-left: 5px solid #E67E22;
    padding: 20px;
    border-radius: 8px;
    margin: 20px 0;
">

**Interactive Mode :**  
Use the widget-based interface for parameter tuning and experimentation.

**Features :**
- Real-time parameter selection (model type, n_estimators, target variable)
- Live training progress and performance feedback
- Quick iteration for hyperparameter optimization
- Automatic logging and artifact saving

**When to use :**
- Exploratory analysis and model comparison
- Fine-tuning hyperparameters
- Testing different prediction targets
- Educational purposes / demonstrations

</div>

In [ ]:
# ============================================================================
# INTERACTIVE TRAINING UI — Widget-Based Interface
# ============================================================================

pipeline.interactive_training_runner()

<div style="height: 50px;"></div>

<hr style="
    border: 0;
    height: 3px;
    background: linear-gradient(90deg, transparent, #2ECC71, transparent);
    border-radius: 2px;
">

<div style="height: 50px;"></div>

<a id="step-1"></a>

# Step 1 : Download Spectra

<div style="
    background: linear-gradient(135deg, #1a1a2e 0%, #16213e 100%);
    border-left: 5px solid #2ECC71;
    padding: 20px;
    border-radius: 8px;
    margin: 20px 0;
">

**Purpose :**  
Download LAMOST DR5 spectral FITS files from the DR5 archive.

**Implementation :**  
This step is handled by the external script `dr5_downloader.py`, which is documented in the companion notebook:
- **[01_download_spectra.ipynb](./01_download_spectra.ipynb)**

**Key Features :**
- Parallel downloads with progress tracking
- Automatic retry on network failures
- MD5 checksum verification
- Organized directory structure by observation date

**Important :**
- Large downloads may take hours depending on network speed
- Ensure sufficient disk space (each spectrum ~50-100 KB)
- Respect LAMOST download quotas and policies

</div>

<div style="
    background: linear-gradient(135deg, #1a1a2e 0%, #16213e 100%);
    border: 1px solid #4A90E2;
    padding: 15px;
    border-radius: 8px;
    margin: 20px 0;
    text-align: center;
">

**💡 Tip :** For this notebook, ensure your `RAW_DATA_DIR` contains downloaded FITS files before proceeding to Step 2.

</div>

<div style="height: 30px;"></div>

<a id="step-2"></a>

# Step 2 : Batch Selection

<div style="
    background: linear-gradient(135deg, #1a1a2e 0%, #16213e 100%);
    border-left: 5px solid #2ECC71;
    padding: 20px;
    border-radius: 8px;
    margin: 20px 0;
">

**Purpose :**  
Select a **new batch** of spectral FITS files for processing, ensuring no overlap with previously used spectra.

**How It Works :**
1. Scans `RAW_DATA_DIR` for available `.fits.gz` files
2. Checks against `DatasetBuilder` logs to identify unused spectra
3. Selects batch according to specified strategy (random, sequential, stratified)
4. Updates internal tracking to prevent duplication

**Parameters :**
- `batch_size` : Number of spectra to select (e.g., 500, 5000, 10000)
- `strategy` : Selection method (`"random"`, `"sequential"`, `"stratified"`)

**Output :**
- List of selected FITS file paths
- Updated `pipeline.current_batch` attribute
- Logging entry in dataset tracking system

</div>

In [ ]:
# ============================================================================
# BATCH SELECTION — Selecting Fresh Spectra
# ============================================================================

print("\nSelecting batch of spectra...\n")

pipeline.select_batch(
    batch_size=1000,     # Adjust based on computational resources
    strategy="random"     # "random", "sequential", or "stratified"
)

print("\n✓ Batch selection complete")
print(f"   → {len(pipeline.current_batch)} spectra selected")
print(f"   → Ready for catalog generation")

<div style="height: 30px;"></div>

<a id="step-3"></a>

# Step 3 : Catalog Generation & Gaia Enrichment

<div style="
    background: linear-gradient(135deg, #1a1a2e 0%, #16213e 100%);
    border-left: 5px solid #2ECC71;
    padding: 20px;
    border-radius: 8px;
    margin: 20px 0;
">

**Purpose :**  
Generate a master catalog from selected FITS files and optionally enrich with Gaia DR3 cross-matching.

**Workflow :**
1. **Local Catalog Creation** : Extract metadata from FITS headers (RA, Dec, SNR, class, magnitude, etc.)
2. **Gaia Cross-Matching** (optional) : Query Gaia DR3 for astrometric and photometric parameters
3. **Quality Filtering** : Apply RUWE cuts and parallax quality checks
4. **Catalog Export** : Save enriched catalog as CSV

**Gaia Parameters Available :**
- **Astrometry** : parallax, proper motions (pmra, pmdec), RUWE
- **Photometry** : G, BP, RP magnitudes and colors
- **Astrophysics** (optional) : effective temperature, radius, mass, age

**Output Files :**
- `master_catalog_temp.csv` — Local LAMOST catalog
- `master_catalog_gaia.csv` — Gaia-enriched catalog (if enabled)

</div>

<div style="
    background: linear-gradient(135deg, #1a1a2e 0%, #16213e 100%);
    border-left: 5px solid #E67E22;
    padding: 15px;
    border-radius: 8px;
    margin: 20px 0;
">

**⚠️ Gaia Connection Required :**  
For `enrich_gaia=True`, ensure valid Gaia credentials in `.env` and stable network connection.

**Best Practices :**
- Use `mode='bulk'` for batches >1000 (more efficient)
- Set `ruwe_max=1.4` to filter poor-quality astrometry
- Keep `include_risky=False` unless you need experimental astrophysical parameters

</div>

In [ ]:
# ============================================================================
# CATALOG GENERATION & GAIA ENRICHMENT
# ============================================================================

print("\nGenerating catalog and enriching with Gaia DR3...\n")

pipeline.generate_and_enrich_catalog(
    # ─────────────────────────────────────────────────────────────────────
    # GAIA ENRICHMENT SETTINGS
    # ─────────────────────────────────────────────────────────────────────
    enrich_gaia=True,          # Enable/disable Gaia cross-matching
    mode='bulk',               # 'bulk' (faster) or 'cone' (individual queries)
    
    # ─────────────────────────────────────────────────────────────────────
    # QUALITY FILTERS
    # ─────────────────────────────────────────────────────────────────────
    ruwe_max=1.4,              # Max RUWE for astrometric quality
    include_risky=False,       # BETA: Include radius/mass/age estimates
)

print("\n✓ Catalog generation complete")
print(f"   → Total entries: {len(pipeline.master_catalog_df)}")
print(f"   → Gaia matches: {pipeline.master_catalog_df['source_id'].notna().sum()}")
print(f"   → Catalog saved to: {pipeline.catalog_dir}")

<div style="height: 30px;"></div>

<a id="step-4"></a>

# Step 4 : Feature Extraction

<div style="
    background: linear-gradient(135deg, #1a1a2e 0%, #16213e 100%);
    border-left: 5px solid #2ECC71;
    padding: 20px;
    border-radius: 8px;
    margin: 20px 0;
">

**Purpose :**  
Apply spectral preprocessing and extract machine learning features from each spectrum.

**Preprocessing Steps :**
1. **Wavelength Calibration** : Convert pixels to Angstroms
2. **Continuum Normalization** : Remove spectral continuum
3. **Noise Filtering** : Sigma clipping and smoothing
4. **Sky Subtraction** : Remove telluric contamination

**Feature Categories :**
- **Spectral Lines** : Balmer series (Hα, Hβ, Hγ, Hδ), Ca II H/K, Mg b, Na D
- **Line Indices** : Equivalent widths, depths, asymmetries
- **Statistical** : Flux percentiles, gradients, derivatives
- **Photometric** : Colors (u-g, g-r, r-i, i-z), absolute magnitudes
- **Astrometric** (if Gaia) : Parallax, proper motions, RUWE

**Output :**
- `features_YYYYMMDDTHHMMSSZ.csv` in `PROCESSED_DIR`
- Updates `pipeline.features_df` in memory
- Feature statistics and completeness report

</div>

<div style="
    background: linear-gradient(135deg, #1a1a2e 0%, #16213e 100%);
    border: 1px solid #4A90E2;
    padding: 15px;
    border-radius: 8px;
    margin: 20px 0;
">

**💡 Performance Note :**  
Feature extraction is the most computationally intensive step. For 10,000 spectra, expect :
- ~5-15 minutes on modern CPU (8+ cores)
- ~100-500 MB memory usage
- Parallelization available for large batches

</div>

In [ ]:
# ============================================================================
# SPECTRAL PREPROCESSING & FEATURE EXTRACTION
# ============================================================================

print("\nExtracting spectral features...\n")

pipeline.process_data()

print("\nFeature extraction complete")
print(f"   → Features extracted: {pipeline.features_df.shape[1]}")
print(f"   → Spectra processed: {pipeline.features_df.shape[0]}")
print(f"   → Output file: {pipeline.last_features_path}")
print(f"   → Missing values: {pipeline.features_df.isna().sum().sum()}")

<div style="height: 50px;"></div>

<hr style="
    border: 0;
    height: 3px;
    background: linear-gradient(90deg, transparent, #E67E22, transparent);
    border-radius: 2px;
">

<div style="height: 50px;"></div>

<a id="step-5"></a>

# Step 5 : Model Training

<div style="
    background: linear-gradient(135deg, #1a1a2e 0%, #16213e 100%);
    border-left: 5px solid #E67E22;
    padding: 20px;
    border-radius: 8px;
    margin: 20px 0;
">

**Purpose :**  
Train a stellar classification model using the **Interactive Training Interface** (see above) or programmatically via `pipeline.run_training_session()`.

**Training Options :**

1. **Interactive UI** (Recommended for exploration)  
   ↳ Use the widget-based interface above for real-time parameter tuning

2. **Programmatic Training** (For automation/scripting)  
   ↳ Call `pipeline.run_training_session()` directly with your parameters

</div>

<div style="
    background: linear-gradient(135deg, #1a1a2e 0%, #16213e 100%);
    border: 1px solid #4A90E2;
    padding: 15px;
    border-radius: 8px;
    margin: 20px 0;
">

**For detailed training guide and parameter explanations :**  
See documentation at [User Guides → Model Training](https://phd-brown.github.io/AstroSpectro/docs/user-guides/model-training)

</div>

<div style="height: 40px;"></div>

<hr style="
    border: 0;
    height: 3px;
    background: linear-gradient(90deg, transparent, #4A90E2, transparent);
    border-radius: 2px;
">

<div style="height: 40px;"></div>

<div style="
    background: linear-gradient(135deg, #0f2027 0%, #203a43 50%, #2c5364 100%);
    border-radius: 12px;
    padding: 30px;
    text-align: center;
    margin-top: 50px;
    border: 2px solid #4A90E2;
">
    <h2 style="color: #4A90E2; margin: 0; font-size: 24px; font-weight: 300; letter-spacing: 2px;">
        ✨ Pipeline Execution Complete
    </h2>
    <p style="color: #95D5F7; margin-top: 15px; font-size: 14px; line-height: 1.8;">
        All pipeline stages have been successfully executed.<br>
        Model artifacts and training reports are saved in their respective directories.<br>
        <strong>Next steps:</strong> Analyze results, tune hyperparameters, or scale to production.
    </p>
    <div style="margin-top: 25px; padding-top: 20px; border-top: 1px solid rgba(74, 144, 226, 0.3);">
        <p style="color: #95D5F7; font-size: 12px; margin: 0;">
            <strong>AstroSpectro</strong> — Automated Stellar Classification Pipeline<br>
            GitHub: <a href="https://github.com/PhD-Brown/AstroSpectro" style="color: #4A90E2; text-decoration: none;">PhD-Brown/AstroSpectro</a> •
            Docs: <a href="https://phd-brown.github.io/AstroSpectro/" style="color: #4A90E2; text-decoration: none;">phd-brown.github.io/AstroSpectro</a>
        </p>
    </div>
</div>